In [2]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import lxml
import re
requests_cache.install_cache('cache')

## * <span style="color:blue">Web Scrapping</span>  *

We use 3000 posts in Indeed to futher analysis. First, we get the basic information inlcluding location, company name and url for the post. For url, we locate the tags with attribute *<span style="color:blue">"data-tn-element="jobTitle"</span>*, the url is the value for attribute *<span style="color:blue">href</span>*. For location and companies' names, they are the value of tag *<span style="color:blue">span</span>* with attribute *<span style="color:blue">lccation and company </span>* seperately.

In [1]:
def get_url(start_url,pages=1):
    '''
    This function will return url,text  in the desifnated pages.
    Input: start_url: the start page
           pages:number of pages need searching.
    '''
    url=start_url 
    urls=[] #list of urls needed
    loc=[]
    names=[]
    n_next=0 #number of next url
    next_urls=[]
    while True: 
        
        html_doc=requests.get(url)
        content=html_doc.content
        soup = BeautifulSoup(content,"lxml")
        
        #urls
        tags=soup.select('a[data-tn-element="jobTitle"]') #locate
        temp_urls=[tag.get("href") for tag in tags ]
        urls=urls+["https://www.indeed.com"+url for url in temp_urls if ~url.startswith("http")]
        
        #location
        location = soup.find_all(name="span",attrs = {"class":"location"})
        loc =loc+ [i.text for i in location]
        
        #company name
        name_tags=soup.find_all(name="span",attrs = {"class":"company"})
        names=names+[i.text for i in name_tags]
        #combine
        #loc_len=len(loc)
        #url_len=len(urls)
        ###if reach the upper limitation
        if(n_next==pages-1):
            break
        ##get next pages
    
        ##if we can't find next one
        starts=(n_next+1)*10
        next_url="https://www.indeed.com/jobs?q=data+analyst&l=CA&start=%d"%(starts)
        n_next=n_next+1
        url=next_url
        next_urls=next_urls+[url]
        
        
    return urls,loc,names,next_urls
                                                     
    
    

We scarp information from corresponding website according to the url for each post. For the url, we We get job description from body of the website since all the information about the job is the value of tag *<span style="color:blue"> body</span>*.

In [3]:
def get_content(url):
    try:
        doc=requests.get(url)
        job_content=doc.content
        soup_job= BeautifulSoup(job_content,'lxml')
        job_body_tag=soup_job.find_all("body")[0]
        job={"url":url,"text":job_body_tag.text}
    except:
        job={}
    return job

We start from "https://www.indeed.com/jobs?q=data+analyst&l=CA" to find the posts related to data analysts in California. We get 3000 posts in 200 pages.

In [4]:
start_url="https://www.indeed.com/jobs?q=data+analyst&l=CA"

In [5]:
urls,locs,names,nexts=get_url(start_url,pages=200)

In [7]:
names_fn=[x.strip() for x in names]

In [8]:
import collections as cls

In [9]:
cls.Counter(names_fn)#examine names

Counter({u'21Tech': 1,
         u'360 IT Professionals': 1,
         u'39 Celsius Web Marketing': 1,
         u'A9.com': 16,
         u'AAA': 1,
         u'ABHOW': 1,
         u'AECOM': 1,
         u'AIDS Healthcare Foundation': 1,
         u'AJobStaff': 3,
         u'AKQA': 5,
         u'AMERICAN ADVISORS GROUP': 1,
         u'ARES Corporation': 1,
         u'ATR International': 1,
         u'Abacus Service Corporation': 2,
         u'Abaxis': 1,
         u'Able Forces': 1,
         u'Academic Organization': 1,
         u'Accelon Inc': 2,
         u'Accenture': 1,
         u'Accountemps': 1,
         u'Activision': 1,
         u'Acuitus': 2,
         u'Acumen LLC': 1,
         u'Ad Blot, Inc.': 1,
         u'AdRoll': 1,
         u'Adept Solutions': 3,
         u'Adobe': 4,
         u'Advancement Project': 24,
         u'Adventist Health': 2,
         u'Aetna': 1,
         u'Affinity, Inc.': 1,
         u'Agilent': 1,
         u'Alambic Investment Management, LP': 46,
         u'Alamed

In [10]:
import pandas as pd

In [11]:
job_info=[pd.DataFrame(get_content(url) for url in urls )] #get text for each post

In [12]:
job_info_fn=pd.concat(job_info)

In [13]:
job_info_fn.head()

,text,url
0,\n\n\n\n\n\n\nSAP is committed to the principl...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,\n\n\n\n\n\n\n\n\n\n\nCareers HomeEmployment L...,https://www.indeed.com/rc/clk?jk=84bc38d534438...
4,"Entry Level Business AnalystCalifornia City, C...",https://www.indeed.com/rc/clk?jk=4c0b2bbed6002...


In [14]:
job_info_fn['loc']=locs #add locations

In [15]:
job_info_fn['name']=names_fn #add names

After combining the information about the jobs, we get dataframe.

In [16]:
job_info_fn.head(4)

,text,url,loc,name
0,\n\n\n\n\n\n\nSAP is committed to the principl...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",SAP
1,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",Sun Basket
2,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",Komodo Health
3,\n\n\n\n\n\n\n\n\n\n\nCareers HomeEmployment L...,https://www.indeed.com/rc/clk?jk=84bc38d534438...,"Sunnyvale, CA",Astreya


## * <span style="color:blue">Data Cleaning</span>  *

Our goal is to find the common required or prefered skills for data scientist in California, so we remove the posts without mentioning * <span style="color:blue">analy</span>  *. After subset, we have 2455 observations in the data.

In [ ]:
requirements= job_info_fn['require']

In [ ]:
idx=[(x.find("analy"))>-1 for x in requirements]

In [ ]:
job_analyst=job_info_fn[idx]

In [ ]:
len(job_analyst)

Generally, the skills required are in the descripttions of job requirments. We first find the requirements from the job description. Sometimes, the requirements strat with key words like__* <span style="color:red">require, skill, qualification, qualify</span>  * __, we locate requirements by these words. While some posts have other rare pattern, we just keep the whole description.

In [17]:
#remove new lines
def find_require(text):
    '''
    This function find requirements.
    If key words for requirements are find, we keep the part from key words
    else return the whole description
    '''
    try:
        new=(text.strip()).replace("\n"," ")
        new_low=(new.lower()).translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22, 0x2026:0x20 })
        require=re.search("(require|skill|qualification|qualify).+(apply)?",new_low)
    except:
        return ""
    if require==None:
        return new_low
    else:
        requirement=require.group(0)
    return requirement

In [18]:
job_info_fn['require']=[find_require(x) for x in job_info_fn['text']]

In [19]:
job_info_fn.head()

,text,url,loc,name,require
0,\n\n\n\n\n\n\nSAP is committed to the principl...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",SAP,sap is committed to the principles of equal em...
1,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",Sun Basket,required with a strong preference for an analy...
2,"Skip to Job Description, SearchClose\n\n ...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"San Francisco, CA",Komodo Health,"requirementsqualificationsbs, ms, or phd in co..."
3,\n\n\n\n\n\n\n\n\n\n\nCareers HomeEmployment L...,https://www.indeed.com/rc/clk?jk=84bc38d534438...,"Sunnyvale, CA",Astreya,skills. we need sound data analysis skills and...
4,"Entry Level Business AnalystCalifornia City, C...",https://www.indeed.com/rc/clk?jk=4c0b2bbed6002...,"California City, CA",NetCloudTek,qualificationsbachlors degree. major in financ...


In [24]:
job_analyst.to_csv("data_analyst.csv", sep=',',encoding='utf-8')

In [31]:
# we export data and read it to avoid repeatedly run code
job_analyst=pd.read_csv("C:\\Users\\aran\\Documents\\stat141B\\project\\jobs_analyst.csv")

## * <span style="color:blue">Data processing</span>  *

Initially, we examine the difference of job opportunity is CA. We plot distribution of the posts on the map. We set point size according to the number of posts(jobs) in the city. The larger the size, the more employment opportunity in that city.

In [ ]:
#clean for city
citys=[re.sub('(.+),.*','\\1',x) for x in job_analyst['loc']] #only keep city name

In [ ]:
city_count=cls.Counter(citys).most_common

In [ ]:
city_fn=pd.DataFrame.from_dict(city_count,orient='index')

In [ ]:
city_fn.columns=['counts']

In [ ]:
city_fn['city']=city_fn.index

In [ ]:
city_fn=city_fn.set_index([range(len(city_fn))])

Then, we set size according to the number of posts of the city.

In [ ]:
def city_size(number):
    if number>600:
        return 10
    elif number>200:
        return 6
    elif number>100:
        return 5
    elif number>80:
        return 4
    elif number>50:
        return 3
    elif number>30:
        return 2
    elif number>10:
        return 1
    else:
        return 0.5
        

In [ ]:
city_fn['size']=[city_size(x) for x in city_fn['counts']]

city_fn

In [ ]:
#map

First, we figure out what is the min of working experience required and what will be the lowest degree

We get years of experience and degree in the requirment by regular expression. For years of experience, generally, it comes with years following. Then pattern is like __* <span style="color:red">x-x years</span>  * __ or __* <span style="color:red">x years</span>  * __. Since years mentioned in the job description are close and the one mentioned first is likely to be the minumum working exprience, we choose the first one as years of experience. When it comes for degree, we also choose the first one for the reason that the first mentioned degree will be the minumum requirement.

In [32]:
job_analyst['year']=[re.sub('(.+)([0-9]{1,2}-? *[0-9]{1,2}.*year.{0,30})(.*)','\\2',x) for x in (job_analyst['require_fn'])]

In [34]:
def get_year(text):
    '''
    This function get year from text.
    '''
    year_temp=re.match("[0-9]+",text)
    if year_temp==None:
        return 0
    else:
        return year_temp.group(0)

In [39]:
#exp_job['year_fn']=[get_year(x) for x in exp_job['year']]
job_analyst['year']=[get_year(x) for x in job_analyst['year']]

In [41]:
max(job_analyst['year'])

8

Then, we group year as 3 groups: less than 2, 3-5 and greater than 5:

In [124]:
def year_group(year):
    '''
    This function group year accroding to the input.
    '''
    if year>5:
        return "greater than 5"
    elif year>3:
        return "3-5"
    else:
        return "less than 2"

In [ ]:
job_analyst["year_group"]=[year_group(year) for year in job_analyst['year']]

Next, we find degree. The  common pattern are: __* <span style="color:red">bachelor, ms, phd, bs, master, ba</span>  * __ . We extract these pattern from the requirement.

In [127]:
job_analyst['degree']=[get_degree(x) for x in job_analyst['require_fn']]

def get_degree(text):
    '''
    This function get degree from text.
    '''
    degree_temp=re.search("bachelor|ms|phd|bs|master|ba",text)
    if degree_temp==None:
        return None
    else:
        degree_temp=degree_temp.group(0)
        if degree_temp=='bs':
            return "bachelor"
        elif degree_temp=="ms":
            return "master"
        else:
            return 'phd'

In [79]:
#process
import numpy as np
import nltk
from nltk import corpus
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## * <span style="color:blue">Natural Languege Processing</span>  *

To find the common required skills, we need to find the idf for each word. Then, we select skills with small idf since the smaller the idf, the more common a certain skill is in these posts.

In [43]:
stemmer = PorterStemmer().stem

tokenize = nltk.word_tokenize

def stem(tokens,stemmer = PorterStemmer().stem):
    return [stemmer(w.lower()) for w in tokens] 

def lemmatize(text):
    """
    Extract simple lemmas based on tokenization and stemming
    Input: string
    Output: list of strings (lemmata)
    """
    return stem(tokenize(text))

vectorizer = TfidfVectorizer(tokenizer=lemmatize,stop_words="english",smooth_idf=True,norm=None)
tfs = vectorizer.fit_transform(exp_job['require'])

## * <span style="color:blue">Caculate idf </span>  *

In [ ]:
words=vectorizer.vocabulary_

In [ ]:
idfs=vectorizer.idf_

In [ ]:
word_df=pd.DataFrame(words.items())

In [ ]:
word_df.columns=['word','pos']

In [ ]:
word_df_pos=word_df.sort("pos")

In [ ]:
word_df_pos['idf']=idfs

In [ ]:
sorted_words=pd.DataFrame(word_df_pos.sort('idf'))#sort according to idf

The skills could only start with letter, so we remove the words do not start with letters

In [ ]:
alpha=[x.isalpha() for x in sorted_words['word']]

In [ ]:
words_alpha=sorted_words[alpha]

We find that sometimes there is no space between two words so we add space to seperate skills.

In [ ]:
job_analyst['require_fn']=[re.sub("(.+?)(sas|python|eview|spss|tableau|access|vba|matlab)(.+?)","\\1 \\2 \\3 ",x) for x in (job_analyst['require'])]

In [104]:
vectorizer = TfidfVectorizer(tokenizer=lemmatize,stop_words="english",smooth_idf=True,norm=None)
tfs = vectorizer.fit_transform(exp_job['require_fn'])

In [106]:
words=vectorizer.vocabulary_

In [107]:
idfs=vectorizer.idf_

In [108]:
word_df=pd.DataFrame(words.items())

In [109]:
word_df.columns=['word','pos']

In [110]:
word_df_pos=word_df.sort("pos")

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [111]:
word_df_pos['idf']=idfs

In [112]:
sorted_words=pd.DataFrame(word_df_pos.sort('idf'))

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [113]:
alpha=[x.isalpha() for x in sorted_words['word']]

In [114]:
words_alpha=sorted_words[alpha]

In [178]:
words_alpha.to_csv("words_alpha.csv", sep=',',encoding='utf-8')

In [13]:
words_alpha=pd.read_csv("words_alpha.csv")

We find that the common ones are: __* <span style="color:red">spss, excel, sql, python,java, hadoop, pig, spark, hive, orac, vba, mysql, sqlserver, matlab, microsoft,access,matlab,stata,sa,tableau</span>  * __ 


In [14]:
#select softwares
soft_idx=[re.match('^(r|spss|excel|sql|python|java|hadoop|pig|spark|hive|oracl|vba|mysql|\
sqlserver|microsoft|access|matlab|stata|sa|tableau)$',x) for x in words_alpha['word']]


In [15]:
soft_idx1=[not x==None for x in soft_idx]

In [16]:
software=words_alpha[soft_idx1]

In [17]:
software=software[['word','idf']]#show common idf 

In [18]:
software

,word,idf
26,r,1.395435
41,sql,1.517182
50,excel,1.577928
80,access,1.708237
227,python,2.103231
266,java,2.289728
486,hadoop,2.937878
504,tableau,2.982875
529,matlab,3.038804
533,spark,3.047693


In [122]:
soft_pattern=software['word'][(software['word']!='r')& (software['word']!='sa')& (software['word']!='sql')& (software['word']!='excel')]

In [123]:
soft_pattern

80         access
227        python
266          java
486        hadoop
504       tableau
529        matlab
533         spark
630     microsoft
925         oracl
1199          vba
1718         hive
2204         spss
2375        mysql
2659          pig
2877        stata
5085    sqlserver
Name: word, dtype: object

In [128]:
job_analyst.to_csv("job_analyst_mar10.csv")

In [11]:
job_analyst=pd.read_csv("job_analyst_mar10.csv")

In [86]:
soft_mention=[[re.search(x,y) for y in job_analyst['require_fn']] for x in soft_pattern]

In [87]:
soft_mention=(pd.DataFrame(soft_mention)).T

In [88]:
soft_mention.columns=soft_pattern

In [89]:
soft_mention['r']=[re.search(" r ",x) for x in job_analyst['require_fn']]

In [90]:
soft_mention['sas']=[re.search("sas",x) for x in job_analyst['require_fn']]

In [134]:
soft_mention['sql']=[re.search(" sql",x) for x in job_analyst['require_fn']]

In [141]:
soft_mention['excel']=[re.search(" excel",x) for x in job_analyst['require_fn']]

In [125]:
soft_mention['java']=[re.search(" java ",x) for x in job_analyst['require_fn']]

In [142]:
soft_mention=pd.DataFrame(soft_mention)

In [143]:
soft_mention=soft_mention.set_index([range(len(soft_mention))])

In [144]:
soft_mention

word,sql,excel,access,python,java,hadoop,tableau,matlab,spark,microsoft,oracl,vba,hive,spss,mysql,pig,stata,sqlserver,r,sas
0,<_sre.SRE_Match object at 0x0000000011235370>,<_sre.SRE_Match object at 0x0000000011DA18B8>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x0000000011B23B28>,None
1,None,None,<_sre.SRE_Match object at 0x00000000120A5370>,<_sre.SRE_Match object at 0x0000000011623098>,<_sre.SRE_Match object at 0x00000000117B8AC0>,<_sre.SRE_Match object at 0x00000000113BE370>,None,<_sre.SRE_Match object at 0x00000000113D65E0>,<_sre.SRE_Match object at 0x00000000113E7780>,None,None,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x000000001192F7E8>,None
2,<_sre.SRE_Match object at 0x0000000011235308>,<_sre.SRE_Match object at 0x0000000011DA1850>,None,None,None,None,<_sre.SRE_Match object at 0x00000000113D2370>,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,<_sre.SRE_Match object at 0x00000000112353D8>,None,None,<_sre.SRE_Match object at 0x0000000011623100>,None,<_sre.SRE_Match object at 0x00000000113BE3D8>,<_sre.SRE_Match object at 0x00000000113D23D8>,None,<_sre.SRE_Match object at 0x00000000113E77E8>,None,None,None,None,None,None,None,None,None,None,None
5,<_sre.SRE_Match object at 0x0000000011235440>,None,None,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x00000000113EE2A0>,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x000000001192F780>,None
6,<_sre.SRE_Match object at 0x00000000112354A8>,None,<_sre.SRE_Match object at 0x00000000120A53D8>,<_sre.SRE_Match object at 0x0000000011623168>,None,<_sre.SRE_Match object at 0x00000000113BE440>,None,None,None,None,None,None,<_sre.SRE_Match object at 0x00000000119561D0>,None,None,None,None,None,None,None
7,<_sre.SRE_Match object at 0x0000000011235510>,<_sre.SRE_Match object at 0x0000000011DA1920>,None,<_sre.SRE_Match object at 0x00000000116231D0>,None,None,<_sre.SRE_Match object at 0x00000000113D2440>,None,None,None,None,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x000000001192F850>,None
8,<_sre.SRE_Match object at 0x0000000011235578>,<_sre.SRE_Match object at 0x0000000011DA1988>,None,<_sre.SRE_Match object at 0x0000000011623238>,None,None,<_sre.SRE_Match object at 0x00000000113D24A8>,None,None,None,None,None,None,None,None,None,None,None,None,None
9,<_sre.SRE_Match object at 0x00000000112355E0>,<_sre.SRE_Match object at 0x0000000011DA19F0>,None,None,None,None,<_sre.SRE_Match object at 0x00000000113D2510>,None,None,<_sre.SRE_Match object at 0x00000000113E2578>,None,None,None,None,None,None,None,None,<_sre.SRE_Match object at 0x000000001192F8B8>,None


In [112]:
#count for total
def binary_change(result):
    '''This function will change the result for match into 0-1.
    If none y=0 and vice versa.
    '''
    if result==None:
        return 0
    else:
        return 1

In [145]:
binary_list=soft_mention.applymap(binary_change)

In [146]:
binary_list.head()

word,sql,excel,access,python,java,hadoop,tableau,matlab,spark,microsoft,oracl,vba,hive,spss,mysql,pig,stata,sqlserver,r,sas
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0
2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [147]:
counts_whole=binary_list.apply(np.sum, axis=0)#count for all


In [148]:
counts_whole=pd.DataFrame(counts_whole)

In [151]:
counts_whole.columns=['counts']

In [153]:
counts_whole.sort('counts',ascending=False)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,counts
word,
r,1391
excel,1210
sql,1067
access,981
python,750
tableau,310
hadoop,304
spark,261
microsoft,259


In [155]:
counts_whole.to_csv("counts_all.csv")

In [100]:
binary_list['degree']=job_analyst['degree']

binary_list['year_group']=job_analyst['year_group']

In [101]:
binary_list.head()

word,sql,excel,access,python,java,hadoop,tableau,matlab,spark,microsoft,...,hive,spss,mysql,pig,stata,sqlserver,r,sas,degree,year_group
0,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,phd,less than 2
1,1,0,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,1,0,bachelor,less than 2
2,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,master,less than 2
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,phd,less than 2
4,1,0,0,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,phd,3-5
